In [1]:
import pandas as pd

# Read dataframe with extracted information
df_niere = pd.read_csv("../metadata/data-dive-files/niere.csv", escapechar='\\')

print df_niere.shape
df_niere.head()

(4454, 14)


,num,filename,author,fulltitle,subtitle,pages,volume,number,keywords,classifications,useful,searchquery_terms,year,abstract
0,1,NaN,[],Selected Characteristics of Campus-Bound Black...,Chronicle of Higher Education,NaN,NaN,NaN,"['Ausschluss,Publikationsart','Quelle,Niere,eb...",[],0,['Niere'],NaN,; A chart is presented that lists the characte...
1,2,NaN,[],"Temsirolimus: CCI 779, CCI-779, Cell cycle inh...",Drugs in R and D,NaN,NaN,NaN,"['Ausschluss,Thema','Quelle,Niere,ovid,systema...",[],0,['Niere'],NaN,. The aim of this study was to determine the e...
2,3,NaN,[],Bevacizumab (Avastin),The Medical letter on drugs and therapeutics,NaN,NaN,NaN,"['Ausschluss,Thema','Quelle,Niere,ovid,systema...",[],0,['Niere'],NaN,NaN
3,4,NaN,[],AE 941,Drugs in R and D,NaN,NaN,NaN,"['Ausschluss,Thema','Quelle,Niere,ovid,systema...",[],0,['Niere'],NaN,12 months. The US FDA granted orphan drug stat...
4,5,NaN,[],'A milestone' in treatment for renal cell cancer,"European journal of cancer (Oxford, England : ...",NaN,NaN,NaN,"['Ausschluss,NKS','Quelle,Niere,ovid,systemati...",[],0,['Niere'],NaN,NaN


In [2]:
# Select fulltitle and abstract of records
df_niere_valid = df_niere.ix[(df_niere.fulltitle.notnull()) & (df_niere.abstract.notnull()),
                            ["num","fulltitle","abstract","keywords","useful"]]

print df_niere_valid.shape
df_niere_valid.head()

(4334, 5)


,num,fulltitle,abstract,keywords,useful
0,1,Selected Characteristics of Campus-Bound Black...,; A chart is presented that lists the characte...,"['Ausschluss,Publikationsart','Quelle,Niere,eb...",0
1,2,"Temsirolimus: CCI 779, CCI-779, Cell cycle inh...",. The aim of this study was to determine the e...,"['Ausschluss,Thema','Quelle,Niere,ovid,systema...",0
3,4,AE 941,12 months. The US FDA granted orphan drug stat...,"['Ausschluss,Thema','Quelle,Niere,ovid,systema...",0
6,7,5HT3-receptor antagonists as antiemetics in ca...,Effective antiemetic therapy is crucial for pa...,"['Ausschluss,Thema','Quelle,Niere,ovid,systema...",0
8,9,Comprehensive side-effect profile of anastrozo...,; p=00004). Interpretation: Anastrozole is tol...,"['Ausschluss,Thema','Quelle,Harnwege,ovid,syst...",0


In [3]:
pd.crosstab(df_niere_valid.useful, columns="freq", normalize=True)

col_0,freq
useful,
0,0.850485
1,0.149515


In [4]:
# 2) From strings to numerical vectors
# https://radimrehurek.com/gensim/tut1.html#from-strings-to-vectors

import nltk
import gensim

def tokenize(text):
    stopwords = set(nltk.corpus.stopwords.words("english"))
    return [x for x in gensim.utils.tokenize(text, lowercase=True, deacc=True, errors="ignore")
           if x not in stopwords]

df_niere_valid.loc[:,"fulltitle_tokens"] = df_niere_valid["fulltitle"].map(lambda x: tokenize(x))  
df_niere_valid.loc[:,"abstract_tokens"] = df_niere_valid["abstract"].map(lambda x: tokenize(x)) 

df_niere_valid.to_csv("../metadata/niere-example/df_classification_valid.csv", index=0)
df_niere_valid.head()

,num,fulltitle,abstract,keywords,useful,fulltitle_tokens,abstract_tokens
0,1,Selected Characteristics of Campus-Bound Black...,; A chart is presented that lists the characte...,"['Ausschluss,Publikationsart','Quelle,Niere,eb...",0,"[selected, characteristics, campus, bound, bla...","[chart, presented, lists, characteristics, bla..."
1,2,"Temsirolimus: CCI 779, CCI-779, Cell cycle inh...",. The aim of this study was to determine the e...,"['Ausschluss,Thema','Quelle,Niere,ovid,systema...",0,"[temsirolimus, cci, cci, cell, cycle, inhibitor]","[aim, study, determine, effect, temsirolimus, ..."
3,4,AE 941,12 months. The US FDA granted orphan drug stat...,"['Ausschluss,Thema','Quelle,Niere,ovid,systema...",0,[ae],"[months, us, fda, granted, orphan, drug, statu..."
6,7,5HT3-receptor antagonists as antiemetics in ca...,Effective antiemetic therapy is crucial for pa...,"['Ausschluss,Thema','Quelle,Niere,ovid,systema...",0,"[ht, receptor, antagonists, antiemetics, cancer]","[effective, antiemetic, therapy, crucial, pati..."
8,9,Comprehensive side-effect profile of anastrozo...,; p=00004). Interpretation: Anastrozole is tol...,"['Ausschluss,Thema','Quelle,Harnwege,ovid,syst...",0,"[comprehensive, side, effect, profile, anastro...","[p, interpretation, anastrozole, tolerated, be..."


In [5]:
# get dictionary from fulltitle string
from gensim import corpora

stoplist = set(nltk.corpus.stopwords.words("english"))
documents = list(df_niere_valid["fulltitle"])


# 2a) Tokenize documents: preprocess the strings by removing stopwords, lowering case.

texts = [[word for word in document.lower().split() if word not in stoplist] for document in documents]
           
# 2b) Dictionary token-to-id: only tokens with frequency > 1 into a dictionary
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
         for text in texts]

# final dictionary
dictionary = corpora.Dictionary(texts)
dictionary.save('../metadata/niere-example/fulltitle.dict')
print dictionary

Dictionary(3363 unique tokens: [u'limited', u'kaplan-meier', u'non-nursing', u'neurofibromatosis', u'hiv-aids']...)


In [6]:
# checking tokens
dictionary.token2id

{u'limited': 42,
 u'kaplan-meier': 1442,
 u'non-nursing': 3303,
 u'neurofibromatosis': 1542,
 u'hiv-aids': 1540,
 u'dynamic': 1330,
 u'four': 570,
 u'obstruction': 3290,
 u'sleep': 357,
 u'neoplasia': 1735,
 u'constants': 3121,
 u'localized': 943,
 u'patient-reported': 678,
 u'relationships': 920,
 u'fibrofolliculomas': 2033,
 u'therapeutics': 512,
 u'medullary': 2061,
 u'physiological': 1461,
 u'containing': 3209,
 u'mediated': 3241,
 u'cm:': 957,
 u'soft-tissue': 100,
 u'ucp2': 714,
 u'magnetic': 2021,
 u'cytochrome': 2781,
 u'updated': 541,
 u'hormone': 434,
 u'risk': 173,
 u'downstream': 1051,
 u'activation': 1285,
 u'nodules': 2113,
 u"women's": 1571,
 u'(arry-142886)': 249,
 u'updates': 1587,
 u'suggests': 3120,
 u'cytokine:': 3208,
 u'affect': 1321,
 u'fractal': 2786,
 u'anemia': 811,
 u'ochratoxin': 1205,
 u'differentiated': 1611,
 u'pooled': 2210,
 u'progression': 538,
 u'induction': 1816,
 u'marechal': 1083,
 u'solution': 1343,
 u'000': 2874,
 u'present,': 1603,
 u'cholestero

In [7]:
# 2c) Convert tokenized documents from (a) to vector, using dictionary in (b) into a corpus
# tokens not found on dict will be ignored
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('../metadata/niere-example/fulltitle.mm', corpus)
